#### Context Question Answering

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nastyachizhikova/doc_test/blob/main/source/notebooks/NER.ipynb)

# Table of contents 

1. [Introduction to the task](#1.-Introduction-to-the-task)

2. [Get started with the model](#2.-Get-started-with-the-model)

3. [Use the model for prediction](#3.-Use-the-model-for-prediction)

    3.1. [Predict using Python](#3.1-Predict-using-Python)
    
    3.2. [Predict using Python pipeline](#3.2-Predict-using-Python-pipeline)
    
    3.3. [Predict using CLI](#3.3-Predict-using-CLI)
     
4. [Train the model on your data](#4.-Train-the-model-on-your-data)
    
    4.1. [Train your model from Python](#4.1-Train-your-model-from-Python)
    
    4.2. [Train your model from CLI](#4.2-Train-your-model-from-CLI)
    
5. [Models list](#5.-Models-list)

# 1. Introduction to the task

Context Question Answering is a task of finding a fragment with an answer to a question in a given segment of context.

For example:

| Context | Question | Answer |
| --- | --- | --- |
| In meteorology, precipitation is any product of the condensation of atmospheric water vapor that falls under gravity. The main forms of precipitation include drizzle, rain, sleet, snow, graupel and hail… Precipitation forms as smaller droplets coalesce via collision with other rain drops or ice crystals **within a cloud**. Short, intense periods of rain in scattered locations are called “showers”. | Where do water droplets collide with ice crystals to form precipitation? | within a cloud |

Here you can see how the answer to the question 'Where do water droplets collide with ice crystals to form precipitation?' can be extracted from the context.

Datasets that follow this task format:

- Stanford Question Answering Dataset (SQuAD) (EN)

- SDSJ Task B (RU)

# 2. Get started with the model

First make sure you have the DeepPavlov Library installed.
[More info about the first installation](https://deeppavlov-test.readthedocs.io/en/latest/notebooks/Get%20Started%20with%20DeepPavlov.html)

In [1]:
!pip install --q deeppavlov

Then make sure that all the required packages for the model are installed.

In [ ]:
!python -m deeppavlov install squad_ru_bert

`squad_ru_bert` here is the name of the model's *config_file*. [What is a Config File?](https://deeppavlov-test.readthedocs.io/en/latest/notebooks/Config%20File.html) 

Configuration file defines the model and describes its hyperparameters. To use another model, change the name of the *config_file* here and further.
The full list of NER models with their config names can be found in the [table](#4.-Models-list).


# 3. Use the model for prediction

## 3.1 Predict using Python

After [installing](#2.-Get-started-with-the-model) the model, build it from the config and predict.

In [ ]:
from deeppavlov import configs, build_model

model = build_model(configs.squad.squad_ru_bert, download=True)

**Input**: List with the context, list with the question

In [ ]:
model(['DeepPavlov is library for NLP and dialog systems.'], ['What is DeepPavlov?'])

[[['Bob', 'Ross', 'lived', 'in', 'Florida'],
  ['Elon', 'Musk', 'founded', 'Tesla']],
 [['B-PERSON', 'I-PERSON', 'O', 'O', 'B-GPE'],
  ['B-PERSON', 'I-PERSON', 'O', 'B-ORG']]]

## 3.2 Predict using Python pipeline

Alternatively, you can use a Python way to describe and build your model for prediction, without using the config file.

In [1]:
from pathlib import Path

from deeppavlov import Element, Model
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
from deeppavlov.download import download_resource
from deeppavlov.models.classifiers.proba2labels import Proba2Labels
from deeppavlov.models.preprocessors.torch_transformers_preprocessor import TorchTransformersNerPreprocessor
from deeppavlov.models.torch_bert.torch_transformers_sequence_tagger import TorchTransformersSequenceTagger


transformer = "bert-base-cased"
model_path = Path('./squad_ru_bert/' + transformer)

download_resource(
    'http://files.deeppavlov.ai/v1/squad/ner_ontonotes_bert_torch.tar.gz',
    {'./ner_ontonotes_bert_torch'}
)

preprocessor = TorchTransformersNerPreprocessor(
    vocab_file=transformer,
    do_lower_case=False,
    max_seq_length=512,
    max_subword_length=15,
    token_masking_prob=0.0,
)
 
classes_vocab = SimpleVocabulary(
    save_path=model_path/'tag.dict',
    load_path=model_path/'tag.dict',
    pad_with_zeros=True,
    unk_token=["O"]
)

tagger = TorchTransformersSequenceTagger(
    n_tags=classes_vocab.len,
    return_probas=False,
    use_crf=True,
    attention_probs_keep_prob=0.5,
    encoder_layer_ids=[-1],
    pretrained_bert='bert-base-cased',
    save_path=model_path/'model',
    load_path=model_path/'model',
    optimizer='AdamW',
    optimizer_parameters={'lr': 2e-05, 
                          "weight_decay": 1e-06, 
                          "betas": [0.9, 0.999],
                          "eps": 1e-06},
    clip_norm=1.0,
    min_learning_rate=1e-07,
    learning_rate_drop_patience=30,
    learning_rate_drop_div=1.5,
    load_before_drop=True,
)

ner_model = Model(
    x=['x'],
    out=["x_tokens", "y_pred"],
    pipe=[
        Element(component=preprocessor, x=['x'], out=["x_tokens", "x_subword_tokens", "x_subword_tok_ids", "startofword_markers", "attention_mask"]),
        Element(component=classes_vocab, x=["y"], out=["y_ind"]),
        Element(component=tagger, x=["x_subword_tok_ids", "attention_mask", "startofword_markers"], out=["y_pred_ind"]),
        Element(component=classes_vocab, x=["y_pred_ind"], out=["y_pred"])
    ]
)

In [ ]:
ner_model(['Bob Ross lived in Florida', 'Elon Musk founded Tesla'])

[[['Bob', 'Ross', 'lived', 'in', 'Florida'],
  ['Elon', 'Musk', 'founded', 'Tesla']],
 [['B-PERSON', 'I-PERSON', 'O', 'O', 'B-GPE'],
  ['B-PERSON', 'I-PERSON', 'O', 'B-ORG']]]

## 3.3 Predict using CLI

You can also get predictions in an interactive mode through CLI.

In [2]:
! python deeppavlov interact squad_ru_bert [-d]

`-d` is an optional download key (alternative to `download=True` in Python code). The key `-d` is used to download the pre-trained model along with embeddings and all other files needed to run the model. 

Or make predictions for samples from *stdin*.

In [ ]:
! python deeppavlov predict squad_ru_bert -f <file-name>

# 4. Train the model on your data


## 4.1 Train your model from Python

### Provide your data path

To train the model on your data, you need to change the path to the training data in the *config_file*. 

Parse the *config_file* and change the path to your data from Python.

In [ ]:
from deeppavlov import configs, train_model
from deeppavlov.core.commands.utils import parse_config

model_config = parse_config(configs.squad.squad_ru_bert)

#  dataset that the model was trained on
print(model_config['dataset_reader']['data_path'])

~/.deeppavlov/downloads/ontonotes/


Provide a *data_path* to your own dataset. 

In [7]:
# download and unzip a new example dataset
!wget http://files.deeppavlov.ai/deeppavlov_data/conll2003_v2.tar.gz
!tar -xzvf "conll2003_v2.tar.gz"

In [6]:
# provide a path to the train file
model_config["dataset_reader"]["data_path"] = "contents/train.txt"


### Train dataset format

To train the model, you need to have a txt-file with a dataset in the following format:

The source text is **tokenized** and **tagged**. For each token, there is a tag with BIO markup. Tags are separated from tokens with **whitespaces**. Sentences are separated with **empty lines**.


### Train the model using new config

In [ ]:
ner_model = train_model(model_config)

Use your model for prediction.

In [ ]:
ner_model(['Bob Ross lived in Florida', 'Elon Musk founded Tesla'])

[[['Bob', 'Ross', 'lived', 'in', 'Florida'],
  ['Elon', 'Musk', 'founded', 'Tesla']],
 [['B-PERSON', 'I-PERSON', 'O', 'O', 'B-GPE'],
  ['B-PERSON', 'I-PERSON', 'O', 'B-ORG']]]

## 4.2 Train your model from CLI

In [ ]:
! python -m deeppavlov train squad_ru_bert

# 5. Models list

The table presents a list of all of the NER-models available in DeepPavlov Library.

| Config name  | Dataset | Language | Model Size | F1 score |
| :--- | --- | --- | --- | ---: |
| [ner_ontonotes_bert](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_ontonotes_bert.json) | Ontonotes | En | 1.3 GB | 87.9 |
| [ner_ontonotes_bert_probas](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_ontonotes_bert_probas.json)| ? | ? |
| [ner_ontonotes_bert_mult](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_ontonotes_bert_mult.json)| Ontonotes | Multi | 2.0 GB | 87.2 |
| [ner_rus_bert](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_bert.json)| Collection3   | Ru | 2.0 GB | 97.7 |
| [ner_rus_bert_probas](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_bert_probas.json)| Collection3   | Ru | ? | ? |
| [ner_rus_convers_distilrubert_2L](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_convers_distilrubert_2L.json)| Collection3   | Ru | ? | ? |
| [ner_rus_convers_distilrubert_6L](https://github.com/deepmipt/DeepPavlov/blob/dev/deeppavlov/configs/ner/ner_rus_convers_distilrubert_6L.json)| Collection3   | Ru |? | ? |

<html>
<table>
<thead>
  <tr>
    <td>ex 1</td>
    <td>ex 2</td>
  </tr>
  <tr>
    <td>ex 1</td>
    <td>ex 2</td>
  </tr>
</thead>
  <tr>
    <td>ex 1</td>
    <td>ex 2</td>
  </tr>
</table>
</html>